In [1]:
from __future__ import division
from pylab import *
from scipy import optimize
import numpy as np
import pandas as pd

import Jesse_Funcs_New.JessePlot
import Jesse_Funcs_New.blahut as blahut
import Jesse_Funcs_New.helpers as helpers

In [2]:
#==============================================================================
#%% Load and preformat the data
#==============================================================================
data_path = 'Data/Partial_Reset_PCM.pkl'
data = pd.read_pickle(data_path)
V = data['V']
R = log10(data['R'])
Pyx, x, y = blahut.Q(V, R, nx=2000, ny=2000)

In [3]:
# x_range = (min(x),max(x))
# y_range = (min(y),max(y))

In [4]:
# #==============================================================================
# # %% Discrete inputs
# #==============================================================================

# filename = 'optimal_discrete_inputs_test_read_write_even_read.npz'

# nbits = arange(1, 9)

# #Initialize the File
# try:
#     data = load(filename)
# except IOError:
#     totsize = nbits.size
#     for var in ['C_in', 'C_out']:
#         locals()[var] = zeros(totsize)
#     for var in ['x_in', 'y_div', 'Px_in', 'Px_out']:
#         locals()[var] = [[0 for j in range(i)] for i in range(totsize)]
#     data = dict(C_in=C_in, x_in=x_in, Px_in=Px_in, nbits=nbits)
#     savez(filename, **data)    

In [5]:
# for i in nbits:
#     idx = i
#     states = 2**(idx + 1)
#     print ('idx: ',idx, ' | states: ', states)

In [6]:
# new_x0 = True
# ny = 32+1


# #Search
# # for repeat in arange(100):
# #     idx = np.random.choice(nbits) - 1
# for idx in range(4):
#     states = 2**(idx + 1)

#     if new_x0 or new_run:
#         x0 = rand(states) * abs(x_range[1]-x_range[0]) + x_range[0]
#         x0.sort()
#     else:
#         x0 = data['x_in'][idx]
    
        
#     bounds = [x_range for i in arange(states)]
#     params = (Pyx, x, y, idx)
    
     
#     minimizer_kwargs = dict(method='L-BFGS-B',
#                             args=params,
#                             bounds=bounds)
        
    
#     def objective(val, Pyx=Pyx, x=x, y=y, idx=idx):
#         xinputs = val
#         xinputs.sort()
        
#         ymin = min(y)
#         ymax = max(y)
#         ydividers = linspace(ymin, ymax, ny+1)
#         ydividers = ydividers[1:-1] 
        
        
#         Pyx_sub, x_sub, y_sub = blahut.quantize(Pyx, x, y, xinputs, ydividers=ydividers)
# #         print ('x_sub: ', x_sub.shape, 'y_sub: ', y_sub.shape) #should be num write by num read (think was doing max read states before [greater than 64])
        
#         C, Px = blahut.blahut_arimoto(Pyx_sub, 
#                                       tolerance=1e-4, 
#                                       iterations = 100)
                                
#         result = {'C_in':C, 'Px_in':Px, 'x_in':xinputs}
#         helpers.add_to_database( result=result, index=idx, 
#                                  goal=('C_in', 'max'), filename=filename )

# #         print ('\nC:', C)
# #         #print ('x_in:', xinputs)
# #         print ('idx:', idx)

#         return -C
    

#     result = optimize.basinhopping(objective, 
#                                    x0=x0, 
#                                    minimizer_kwargs=minimizer_kwargs,
#                                    niter=1000, #was 100
#                                    stepsize=0.5,
#                                    T=0.5)
    
#     print (result)

In [7]:
#==============================================================================
# %% Nonuniform Spacing
#==============================================================================
nx = 5
ny = 5
nbits=4
C_nonequal = zeros((nbits, nbits))
x_in = [ [0 for i in range(nbits)] for j in range(nbits)]
y_out = [ [0 for i in range(nbits)] for j in range(nbits)]
Px_in = [ [0 for i in range(nbits)] for j in range(nbits)]

In [8]:
states = 2**(arange(nbits)+1)

xy = meshgrid(states, states)

xr = [min(x), (min(x) + max(x))/2, max(x)]
yr = [min(y), (min(y) + max(y))/2, max(y)]

In [9]:
states

array([ 2,  4,  8, 16])

In [10]:
C_nonequal

array([[ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.]])

In [11]:
data_file = np.load('C_nonequal_optimized.npz')
data_file['data'].item()['C_nonequal']

array([[ 1.010388  ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  1.59021818,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

In [12]:
def objective(val, Pyx=Pyx, x=x, y=y, nx=nx, ny=ny):
    global C_nonequal, x_in, y_out, Px_in
    
    xinputs = val[0:nx]
    ydividers = val[nx:]
    
    xinputs.sort()
    ydividers.sort()
        
    Pyx_sub, x_sub, y_sub = blahut.quantize(Pyx, x, y, xinputs, ydividers)
#     print ('x_sub: ', x_sub.shape, 'y_sub: ', y_sub.shape) 
    
    C, Px = blahut.blahut_arimoto(Pyx_sub, 
                                  tolerance=1e-7, 
                                  iterations = 1000)
    
    if C > C_nonequal[ix, iy]:        
        C_nonequal[ix, iy] = C 
        x_in[ix][iy] = xinputs
        y_out[ix][iy] = ydividers
        Px_in[ix][iy] = Px

        data = dict(C_nonequal=C_nonequal,
                    x_in = x_in, 
                    y_out = y_out,
                    Px_in = Px_in)
        savez('C_nonequal_optimized.npz', data=data)    
                                    
    
#     print '\nC:', C
#     print 'x:', nx, xinputs
#     print 'y:', ny, ydividers

    return -C


def accept_test(f_new, x_new, f_old, x_old):
    '''
    Make it so there aren't two dividers with the same value
    '''
    if len(x_new) == len(set(x_new)):
        return True
    else:
        return False



#%%

def find_peaks(Px):
    peak = logical_and( logical_and(Px > append(0, Px[:-1]), Px > append(Px[1:],0)), Px > 2e-4)
    return where(peak)


# Px0 = load('./../npz/CEnergy.npz')['data'].item()['Px_list'][0]
# x0_max = x[find_peaks(Px0)]

# y0_max = load('./../npz/basinhopping.npz')['data'].item()['y_div'][10]


for it in range(100):
    print (it)
    for ix, iy in [ (0,0), (0,1), (1,0), (1,1), (1,2), (2,1), (2,2)]:
        ny, nx = (xy[0][ix,iy], xy[1][ix,iy])
    
        xinputs = rand(nx) * abs(xr[2]-xr[0]) + xr[0]
        ydividers = rand(ny-1) * abs(yr[2]-yr[0]) + yr[0]    
        xinputs = sort(xinputs)
        ydividers = sort(ydividers)          

        x0 = np.append(xinputs, ydividers)    
        params = (Pyx, x, y, nx, ny)
        bounds = [ (xr[0], xr[2]) for i in arange(nx) ]
        bounds.extend( [(yr[0], yr[2]) for i in arange(ny-1)] )
            
        minimizer_kwargs = dict(method='L-BFGS-B',
                                args=params,
                                bounds=bounds)
    
        result = optimize.basinhopping(objective, 
                                       x0=x0, 
                                       minimizer_kwargs=minimizer_kwargs,
                                       niter=100, #used to be 50
                                       stepsize=0.5,
                                       T=0.5,
                                       accept_test=accept_test)
        
        print (result)
            
            
    
#%%
# close('all')
# matshow(C_equal)
# show()
    

#%%    
data = dict(C_nonequal=C_nonequal,
            x_in = x_in, 
            y_out = y_out,
            Px_in = Px_in)
savez('C_nonequal_optimized.npz', data=data)  

# #%%
# r = load('./../npz//C_nonequal_optimized.npz')['data'].item()
# for name in ['C_nonequal','x_in','y_out','Px_in']:
#     globals()[name] = r[name]




   




0


/home/rzarcone/PCM_Paper/Jesse_Funcs_New/blahut.py:62: RuntimeWarning: invalid value encountered in log2
  d = p * np.log2(p / np.r_[q])
/home/rzarcone/PCM_Paper/Jesse_Funcs_New/blahut.py:62: RuntimeWarning: divide by zero encountered in log2
  d = p * np.log2(p / np.r_[q])
/home/rzarcone/PCM_Paper/Jesse_Funcs_New/blahut.py:62: RuntimeWarning: invalid value encountered in multiply
  d = p * np.log2(p / np.r_[q])
/home/rzarcone/PCM_Paper/Jesse_Funcs_New/blahut.py:62: RuntimeWarning: invalid value encountered in true_divide
  d = p * np.log2(p / np.r_[q])


                        fun: -1.0068268201786976
 lowest_optimization_result:       fun: -1.0068268201786976
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.,  0.,  0.])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 4
      nit: 0
   status: 0
  success: True
        x: array([ 2.07029494,  2.63595239,  5.81197272])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 0
                       nfev: 404
                        nit: 100
                          x: array([ 2.07029494,  2.63595239,  5.81197272])


/home/rzarcone/PCM_Paper/Jesse_Funcs_New/blahut.py:62: RuntimeWarning: divide by zero encountered in true_divide
  d = p * np.log2(p / np.r_[q])


                        fun: -1.0030858787167485
 lowest_optimization_result:       fun: -1.0030858787167485
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.,  0.,  0.,  0.,  0.])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 6
      nit: 0
   status: 0
  success: True
        x: array([ 1.87001272,  2.9305236 ,  4.59912009,  5.26565947,  8.07613331])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 0
                       nfev: 606
                        nit: 100
                          x: array([ 1.87001272,  2.9305236 ,  4.59912009,  5.26565947,  8.07613331])
                        fun: -1.0024586428399744
 lowest_optimization_result:       fun: -1.0024586428399744
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.,  0.,  0.,  0.,  0.])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 6
     

...

                        fun: -1.0050130657304177
 lowest_optimization_result:       fun: -1.0050130657304177
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.,  0.,  0.,  0.,  0.])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 6
      nit: 0
   status: 0
  success: True
        x: array([ 2.42018517,  2.73175796,  3.7945464 ,  3.89620778,  5.97922225])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 0
                       nfev: 606
                        nit: 100
                          x: array([ 2.42018517,  2.73175796,  3.7945464 ,  3.89620778,  5.97922225])
                        fun: -1.0048065919714348
 lowest_optimization_result:       fun: -1.0048065919714348
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.,  0.,  0.,  0.,  0.])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 6
     

                        fun: -1.6049654712079096
 lowest_optimization_result:       fun: -1.6049654712079096
 hess_inv: <11x11 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 12
      nit: 0
   status: 0
  success: True
        x: array([ 1.        ,  1.50928129,  1.67863971,  1.92709491,  2.37653128,
        2.72503949,  2.9009484 ,  2.95      ,  5.10429265,  6.22415124,
        7.06723091])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 0
                       nfev: 1212
                        nit: 100
                          x: array([ 1.        ,  1.50928129,  1.67863971,  1.92709491,  2.37653128,
        2.72503949,  2.9009484 ,  2.95      ,  5.10429265,  6.22415124,
        7.06723091])
                        fun: -1.9847434966696054
 lowest_optimization_re

                        fun: -1.0039165236683241
 lowest_optimization_result:       fun: -1.0039165236683241
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.,  0.,  0.,  0.,  0.])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 6
      nit: 0
   status: 0
  success: True
        x: array([ 2.19119442,  2.73232266,  5.5153296 ,  7.60860394,  8.15649539])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 0
                       nfev: 606
                        nit: 100
                          x: array([ 2.19119442,  2.73232266,  5.5153296 ,  7.60860394,  8.15649539])
                        fun: -1.0027414776627837
 lowest_optimization_result:       fun: -1.0027414776627837
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.,  0.,  0.,  0.,  0.])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 6
     

                        fun: -1.5845497087438809
 lowest_optimization_result:       fun: -1.5845497087438809
 hess_inv: <11x11 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 12
      nit: 0
   status: 0
  success: True
        x: array([ 1.        ,  1.19501073,  1.19550459,  1.21899473,  1.46207965,
        1.64300257,  2.45314968,  2.95      ,  5.27710149,  5.94586884,
        6.39222262])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 0
                       nfev: 1212
                        nit: 100
                          x: array([ 1.        ,  1.19501073,  1.19550459,  1.21899473,  1.46207965,
        1.64300257,  2.45314968,  2.95      ,  5.27710149,  5.94586884,
        6.39222262])
                        fun: -1.8653160120655388
 lowest_optimization_re

                        fun: -1.0004374847890047
 lowest_optimization_result:       fun: -1.0004374847890047
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.,  0.,  0.,  0.,  0.])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 6
      nit: 0
   status: 0
  success: True
        x: array([ 1.71938891,  2.72240872,  4.0284076 ,  5.6367737 ,  6.26464318])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 0
                       nfev: 606
                        nit: 100
                          x: array([ 1.71938891,  2.72240872,  4.0284076 ,  5.6367737 ,  6.26464318])
                        fun: -1.0094150596726341
 lowest_optimization_result:       fun: -1.0094150596726341
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.,  0.,  0.,  0.,  0.])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 6
     

In [13]:
data_file = np.load('C_nonequal_optimized.npz')
data_file['data'].item()['C_nonequal']

array([[ 1.03066364,  1.03456128,  0.        ,  0.        ],
       [ 1.03526249,  1.98494333,  2.00124845,  0.        ],
       [ 0.        ,  2.00228688,  2.11968304,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

In [ ]:
#==============================================================================
            # %% Equally spaced inputs / outputs Optimized
#==============================================================================

reload(blahut)

nbits=8
C_equal = zeros((nbits, nbits))
C_equal_bounds = zeros((nbits, nbits, 4)) #xmin, xmax, ymin, ymax
states = 2**(arange(nbits)+1)
xy = meshgrid(states, states)

xr = [min(x), (min(x) + max(x))/2, max(x)]
yr = [min(y), (min(y) + max(y))/2, max(y)]


def objective(val, Pyx=Pyx, x=x, y=y, nx=nx, ny=ny):
    global C_equal, C_equal_bounds
    
    xmin, xmax, ymin, ymax = val   

    xinputs = linspace(xmin, xmax, nx)                 
    ydividers = linspace(ymin, ymax, ny+1)
    ydividers = ydividers[1:-1]  
        
    Pyx_sub, x_sub, y_sub = blahut.quantize(Pyx, x, y, xinputs, ydividers)
    C, Px = blahut.blahut_arimoto(Pyx_sub, 
                                  tolerance=1e-7, 
                                  iterations = 100)

    
    if C > C_equal[ix, iy]:        
        C_equal[ix, iy] = C 
        C_equal_bounds[ix, iy, :] = (xmin, xmax, ymin, ymax) 
        data = dict(C_equal=C_equal,
                    C_equal_bounds=C_equal_bounds)
        savez('./../npz/C_equal_optimized.npz', data=data)    
                                    

    
    print '\nC:', C
    print 'nx:', nx, xmin, xmax
    print 'ny:', ny, ymin, ymax

    return -C


def accept_test(f_new, x_new, f_old, x_old):
    '''
    Make it so there aren't two dividers with the same value
    '''
    if len(x_new) == len(set(x_new)):
        return True
    else:
        return False



#%%

for repeats in arange(1000):
    for ix in arange(nbits):
        for iy in arange(nbits):
            ny, nx = (xy[0][ix,iy], xy[1][ix,iy])
    
            params = (Pyx, x, y, nx, ny)
            bounds = [ (xr[0], xr[1]), (xr[1], xr[2]), (yr[0], yr[1]), (yr[1], yr[2]) ]
    
            minimizer_kwargs = dict(method='L-BFGS-B',
                                    args=params,
                                    bounds=bounds)
    
            x0 = array([xr[0], xr[2], yr[0], yr[2]])
        
            result = optimize.basinhopping(objective, 
                                           x0=x0, 
                                           minimizer_kwargs=minimizer_kwargs,
                                           niter=500,
                                           stepsize=1.0,
                                           T=0.5,
                                           accept_test=accept_test)
        
        
    
#%%
close('all')
matshow(C_equal)
show()
    

#%%    
data = dict(C_equal=C_equal,
            C_equal_bounds=C_equal_bounds)
savez('./../npz/C_equal_optimized.npz', data=data)    

#%%
r = load('./../npz/C_equal_optimized.npz')['data'].item()
C_equal= (r['C_equal'])
C_equal_bounds= (r['C_equal_bounds'])
